In [1]:
from mlgrad.risk import ERisk, ERiskGB
from mlgrad.loss import SquareErrorLoss
from mlgrad.model import LinearFuncModel, SigmaNeuronModel, FFNetworkModel, \
                        FFNetworkFuncModel, SigmaNeuronModelLayer, LinearModel
from mlgrad.func import SoftPlus, Sqrt, Sigmoidal, HingeSqrt

from mlgrad import erm_fg, erm_irgd, fg, erm_fg, erisk
from mlgrad.regr import m_regression_irls
from mlgrad.af import averaging_function
from mlgrad.weights import MWeights

import numpy as np

In [2]:
def find_alpha(X, E, mod, W=None):
    Yh = np.array([mod(Xk) for Xk in X])
    if W is None:
        alpha = np.sum(E * Yh) / np.sum(Yh * Yh)
    else:
        alpha = np.sum(W * E * Yh) / np.sum(W * Yh * Yh)
    return alpha

In [3]:
def find_param(X, E, mod, loss_func, W=None):
    rs = ERisk(X, E, mod, loss_func)
    if W is not None:
        rs.use_weights(W)
    alg_fg = erm_fg(rs, h=0.01, tol=1.0e-6)

In [13]:
def find_param_alpha_agg(X, Y, lfm, mod, loss_func, agg, n_iter=30, tol=1.0e-5):
    alpha = alpha_min = 1
    W = np.zeros(len(X), 'd')
    param_min = mod.param.copy()

    L = np.array([loss_func(lfm(Xk) + alpha*mod(Xk), Yk) for Xk, Yk in zip(X, Y)])
    agg.fit(L)
    lval = lval_min = agg.u
    agg.gradient(L, W)
    
    for j in range(n_iter):        
        lval_prev = lval

        E = np.array([(Yk - lfm(Xk))/alpha for Xk, Yk in zip(X,Y)])
        find_param(X, E, mod, loss_func, W)
        
        E = (Y - np.array([lfm(Xk) for Xk in X]))
        alpha = find_alpha(X, E, mod, W)

        if lval < lval_min:
            param_min = mod.param.copy()
            alpha_min = alpha
            lval_min = lval

        L = np.array([loss_func(lfm(Xk) + alpha*mod(Xk), Yk) for Xk, Yk in zip(X, Y)])
        agg.fit(L)
        lval = agg.u
        
        agg.gradient(L, W)
        
        if lval < lval_min:
            param_min = mod.param.copy()
            alpha_min = alpha
            lval_min = lval
            
        if j < 2:
            continue
        
        if abs(lval - lval_prev) < tol:
            break
            
    mod.param[:] = param_min
    alpha = alpha_min
    print(j)
    return alpha
        

In [5]:
def find_param_alpha(X, Y, lfm, mod, loss_func, n_iter=100, tol=1.0e-6):
    lval = lval_min = 10000000.
    alpha_min = alpha = 1

    for j in range(n_iter):
        lval_prev = lval

        E = np.array([(Yk - lfm(Xk)) for Xk,Yk in zip(X,Y)])
        find_param(X, E/alpha, mod, loss_func)
        
        alpha = find_alpha(X, E, mod)

        lval = np.mean([loss_func(lfm(Xk) + alpha*mod(Xk), Yk) for Xk, Yk in zip(X, Y)])

        if lval < lval_min:
            param_min = mod.param.copy()
            alpha_min = alpha
            lval_min = lval

        if j < 2:
            continue
        
        if abs(lval - lval_prev)  < tol:
            break

    mod.param[:] = param_min
    alpha = alpha_min
    print(j)
    return alpha        

In [6]:
def gb_fit_agg(X, Y, alpha=0.75, n_iter=100):
    N = len(X)
    lfm = LinearFuncModel()
    agg = averaging_function('WM', alpha=alpha)
    loss_func = SquareErrorLoss()
    lvals = []
    a = 1
    for k in range(n_iter):
        # body = FFNetworkModel()
        # layer = SigmaNeuronModelLayer(Sigmoidal(), X.shape[1], 3)
        # body.add(layer)
        # head = LinearModel(3)
        # mod = FFNetworkFuncModel(head, body)
        # mod.allocate()
        mod = SigmaNeuronModel(Sigmoidal(), X.shape[1])
        mod.init_param()

        a = find_param_alpha_agg(X, Y, lfm, mod, loss_func, agg)
        # print(k, lfm.weights)
        lfm.add(mod, a)
        # s = np.sum(lfm.weights)
        # lfm.weights = [w/s for w in lfm.weights]
        L = np.array([loss_func(lfm(Xk), Yk) for Xk, Yk in zip(X,Y)])
        agg.fit(L)
        lvals.append(agg.u)
    return lfm, lvals


In [7]:
def gb_fit(X, Y, n_iter=100):
    lfm = LinearFuncModel()
    loss_func = SquareErrorLoss()
    lvals = []
    for k in range(n_iter):
        # body = FFNetworkModel()
        # layer = SigmaNeuronModelLayer(Sigmoidal(), X.shape[1], 3)
        # body.add(layer)
        # head = LinearModel(3)
        # mod = FFNetworkFuncModel(head, body)
        # mod.allocate()
        mod = SigmaNeuronModel(Sigmoidal(), X.shape[1])
        mod.init_param()

        a = find_param_alpha(X, Y, lfm, mod, loss_func)
        lfm.add(mod, a)

        lvals.append(np.mean([loss_func(lfm(Xk), Yk) for Xk, Yk in zip(X,Y)]))
    return lfm, lvals

In [8]:
import sklearn.datasets as datasets

In [9]:
X, Y = datasets.load_diabetes(return_X_y=True)

In [10]:
# Is = np.random.randint(0, len(X), 30)
# for i in Is:
#     Y[i] += 1000

In [11]:
lfm, lvals = gb_fit(X, Y, n_iter=50)
print(np.asarray(lfm.weights))

45
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
[ 3.06894544e+02  1.34861490e+00  9.86430666e-01 -5.43124262e-01
  1.79995361e+00 -8.25175559e-01  1.01009804e+00 -4.12760369e-01
  7.78248084e-01 -4.96263594e-01  7.70907047e-01 -3.21292131e-01
  1.10042108e+00 -4.74315079e-01  8.99283986e-01 -4.90860439e-01
  1.07885134e+00 -6.40946265e-01 -2.38524460e-01  6.52306879e-01
 -3.80891833e-01  6.67182776e-02  1.12162846e+00 -5.43719147e-01
 -4.50777409e-01  3.43475040e-01  7.85554930e-01 -3.59835082e-01
  7.69943167e-01 -5.04309901e-01 -3.15101349e-01  2.53613791e-01
  1.11551243e+00 -5.88308316e-01  9.06763103e-01 -5.42848227e-01
  8.34479639e-01 -5.61751198e-01 -3.22645281e-01  3.49222620e-01
  9.53694585e-01 -5.96297788e-01  8.47939385e-01 -3.71345776e-01
  9.34885975e-01 -4.29489090e-01  8.09845982e-01 -3.09581349e-01
  7.20521350e-01 -4.00152608e-01]


In [12]:
lfm_agg, lvals_agg = gb_fit_agg(X, Y, alpha=0.95, n_iter=50)
print(np.asarray(lfm_agg.weights))

29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29


UnboundLocalError: local variable 'param_min' referenced before assignment

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(np.log(lvals), label='ls')
plt.plot(np.log(lvals_agg), label='wm')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(sorted([abs(lfm(X[k])-Y[k]) for k in range(len(X))]), label='ls')
plt.plot(sorted([abs(lfm_agg(X[k])-Y[k]) for k in range(len(X))]), label='wm')
plt.legend()
plt.show()

In [ ]:
50/len(X)